In [27]:
from metakb.database import Graph

g = Graph(uri="bolt://localhost:7687", credentials=("neo4j", "admin"))
g.clear()

In [28]:
from metakb import PROJECT_ROOT
import json
fpath = PROJECT_ROOT / 'analysis' / 'graph' / 'civic_cdm_updated.json'
with open(fpath, 'r') as f:
    items = json.load(f)

In [ ]:
genes_blacklist = {'civic:gid30', 'civic:gid52'}
therapies_blacklist = {'civic:tid237', 'civic:tid101', 'civic:tid326', 'civic:tid457', 'civic:tid14', 'civic:tid144', 'civic:tid138', 'civic:tid260', 'civic:tid505'}
ev_blacklist = {'civic:eid8190', 'civic:eid2924', 'civic:eid2919', 'civic:eid8187', 'civic:eid8186', 'civic:eid2922', 'civic:eid8184', 'civic:eid2923', 'civic:eid8185', 'civic:eid8189', 'civic:eid1362'}
count = 0
for item in items:
    if 'assertion' in item.keys():
        continue
    if any(filter(lambda x: x['id'] in genes_blacklist, item.get('gene_descriptors', []))):
        continue
    elif any(filter(lambda x: x['id'] in therapies_blacklist, item['therapy_descriptors'])):
        continue
    elif any(filter(lambda x: x['id'] in ev_blacklist, item['evidence'])):
        continue
    else:
        g.add_transformed_data(item)
        count += 1

In [23]:
count

1021